# Vase volumes: basic masking and viewer overlays

This notebook shows how to define a vase (time-varying polygons), build a mask, and use both verb and grammar APIs without changing the default `pipe(cube) | v.plot()` workflow.

In [ ]:
import numpy as np
import xarray as xr
from shapely.geometry import Point

from cubedynamics import pipe, verbs as v
from cubedynamics.vase import VaseDefinition, VaseSection, build_vase_mask
from cubedynamics.plotting import CubePlot

# Synthetic cube for demonstration
time = np.array(['2020-01-01', '2020-06-01', '2020-12-31'], dtype='datetime64[ns]')
y = np.linspace(-1.5, 1.5, 5)
x = np.linspace(-1.5, 1.5, 5)
data = np.random.rand(len(time), len(y), len(x))
cube = xr.DataArray(data, coords={'time': time, 'y': y, 'x': x}, dims=('time', 'y', 'x'), name='value')

# Define a vase that widens over the year
vase_sections = [
    VaseSection(time='2020-01-01', polygon=Point(0, 0).buffer(0.5)),
    VaseSection(time='2020-12-31', polygon=Point(0, 0).buffer(1.1)),
]
vase = VaseDefinition(vase_sections)


In [ ]:
# Baseline: viewer without any vase volume
pipe(cube) | v.plot(title='Baseline cube')


In [ ]:
# Apply the vase mask and render only voxels inside the hull
vase_cube = pipe(cube) | v.vase_extract(vase)
vase_cube | v.plot(title='Inside the vase volume')


In [ ]:
# Use the grammar directly and overlay outlines on cube faces
p = (CubePlot(cube, title='Vase outline overlay')
     .stat_vase(vase)
     .geom_cube()
     .geom_vase_outline(color='limegreen', alpha=0.7)
)

p  # display in notebooks
